In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "X-Linked_Lymphoproliferative_Syndrome"
cohort = "GSE243973"

# Input paths
in_trait_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome"
in_cohort_dir = "../../input/GEO/X-Linked_Lymphoproliferative_Syndrome/GSE243973"

# Output paths
out_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE243973.csv"
out_gene_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE243973.csv"
out_clinical_data_file = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE243973.csv"
json_path = "../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE243973_family.soft.gz', 'GSE243973_series_matrix.txt.gz']
Identified SOFT files: ['GSE243973_family.soft.gz']
Identified matrix files: ['GSE243973_series_matrix.txt.gz']

Background Information:
!Series_title	"Circulating monocyte counts coupled with a 4-gene signature at leukapheresis predict survival of lymphoma patients treated with CAR T"
!Series_summary	"CD19-directed chimeric antigen receptor (CAR) T cells can induce durable remissions in relapsed/refractory large B-cell lymphomas (R/R LBCL), but 60% of patients still relapse. Biological mechanisms explaining lack of disease-response  are largely unknown. To identify mechanisms of response and survival before CAR T manufacturing in 95 R/R LBCL receiving tisagenlecleucel or axicabtagene ciloleucel, we performed phenotypic, transcriptomic and functional evaluations of leukapheresis products (LK).  Transcriptomic profiling of T cells in LK, revealed a signature composed of 4 myeloid genes able to 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List
import gzip

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptomic data of T cells
# and uses nCounter 780 gene CAR-T characterization panel, which indicates gene expression data
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary:
# Key 0 contains disease state (LBCL vs healthy control) - this is our trait
# There's no explicit age information
# There's no explicit gender information
trait_row = 0
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> int:
    """Convert disease state to binary: 1 for LBCL, 0 for healthy control."""
    if value is None:
        return None
    
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "lymphoma" in value.lower() or "large b-cell" in value.lower():
        return 1  # Patient with LBCL
    elif "healthy" in value.lower() or "control" in value.lower():
        return 0  # Healthy control
    else:
        return None  # Unknown or unclassifiable

def convert_age(value: str) -> Optional[float]:
    """Convert age to float. Not used in this dataset as age is not available."""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary. Not used in this dataset as gender is not available."""
    return None

# 3. Save Metadata - Initial Filtering
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the sample characteristics directly from matrix file
    matrix_file_path = os.path.join(in_cohort_dir, "GSE243973_series_matrix.txt.gz")
    
    try:
        # Parse the series matrix file to extract clinical data
        sample_characteristics = {}
        sample_ids = []
        
        with gzip.open(matrix_file_path, 'rt') as f:
            parsing_characteristics = False
            parsing_sample_ids = False
            
            for line in f:
                line = line.strip()
                
                # Extract sample IDs
                if line.startswith('!Sample_geo_accession'):
                    sample_ids = line.split('\t')[1:]
                    parsing_sample_ids = True
                
                # Extract sample characteristics
                elif line.startswith('!Sample_characteristics_ch'):
                    parsing_characteristics = True
                    row_idx = int(line.split('!Sample_characteristics_ch')[1].split('\t')[0]) - 1
                    values = line.split('\t')[1:]
                    
                    if row_idx not in sample_characteristics:
                        sample_characteristics[row_idx] = values
                
                # End of sample information section
                elif parsing_characteristics and not line.startswith('!Sample_'):
                    parsing_characteristics = False
                    
                # If we've passed the sample section, break the loop
                elif parsing_sample_ids and not line.startswith('!') and not line.startswith('#'):
                    break
        
        # Create DataFrame for clinical data
        clinical_data = pd.DataFrame(columns=['ID'])
        clinical_data['ID'] = sample_ids
        
        for row_idx, values in sample_characteristics.items():
            clinical_data[f'characteristic_{row_idx}'] = values
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    
    except Exception as e:
        print(f"Error processing clinical data: {str(e)}")


Preview of selected clinical features:
{'ID': [nan], 'characteristic_0': [1.0]}
Clinical data saved to: ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/clinical_data/GSE243973.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ABCF1', 'ACACA', 'ACAD10', 'ACADVL', 'ACOT2', 'ACSF2', 'ACSL5',
       'ACTN1', 'ACVR1B', 'ACVR1C', 'ACVR2A', 'ADAR', 'ADD1', 'ADORA2A',
       'AFDN', 'AHR', 'AKT1', 'AKT2', 'ALDH1L1', 'ALDH1L2'],
      dtype='object', name='ID')

Gene expression data shape: (794, 85)


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers:
# These identifiers (ABCF1, ACACA, ACAD10, etc.) appear to be standard human gene symbols
# They are in the correct format for human gene symbols (uppercase letters and numbers)
# They represent known human genes (e.g., ABCF1 is ATP Binding Cassette Subfamily F Member 1)
# No mapping to gene symbols is needed as they are already in the correct format

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (762, 85)
First few gene symbols after normalization: ['ABCF1', 'ACACA', 'ACAD10', 'ACADVL', 'ACOT2', 'ACSF2', 'ACSL5', 'ACTN1', 'ACVR1B', 'ACVR1C']
Normalized gene data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/gene_data/GSE243973.csv
Loaded clinical data:
     characteristic_0
ID                   
NaN               1.0
Transposed clinical data to correct format:
ID                NaN
characteristic_0  1.0
Number of common samples between clinical and genetic data: 0
Clinical data index: ['characteristic_0']
Gene data columns: ['GSM7802550', 'GSM7802551', 'GSM7802552', 'GSM7802553', 'GSM7802554', '...']
Extracted 85 GSM IDs from gene data.
Created new clinical data with matching sample IDs:
            X-Linked_Lymphoproliferative_Syndrome
GSM7802550                                      1
GSM7802551                                      1
GSM7802552                                      1
GSM7802553                      

For the feature 'X-Linked_Lymphoproliferative_Syndrome', the least common label is '1' with 14 occurrences. This represents 16.47% of the dataset.
The distribution of the feature 'X-Linked_Lymphoproliferative_Syndrome' in this dataset is fine.

Is trait biased: False


Data quality check result: Usable
Linked data saved to ../../output/preprocess/X-Linked_Lymphoproliferative_Syndrome/GSE243973.csv
